In [1]:
# import packages
from yfapi import YahooFinanceAPI, Interval
from datetime import datetime, timedelta
import psycopg2
from sqlalchemy import create_engine
from decouple import config

In [2]:
# Get database credentials, db path and twitter bearer token from env-file
dbuname = config('DBUNAME')
dbpwd = config('DBPWD')
host = config('HOST')
port = config('PORT')
dbname = config('DBNAME')

In [3]:
# instantiate the API and set the interval to daily data
dh = YahooFinanceAPI(Interval.DAILY)

In [9]:
# define function for fetching and loading the data

def fetch_load_data(instrument, label, start, end, method):
    
    while True:    
        #fetch data
        try:
            df = dh.get_ticker_data(instrument, start, end)
            table_name = 'table_yf_'+labels[instrument]
            print(table_name)
            df['Ins'] = instrument
            df['Ins_label'] = label
        except Exception as e:
            print(e)
            print("Investment Instrument not available: " + instrument)
            break
        
                
        #load data
        if_ex_val = method
        
        try:
            conn_string = 'postgresql://'+dbuname+':'+dbpwd+'@'+host+':'+port+'/'+dbname
            engine = create_engine(conn_string)
            df.to_sql(table_name, conn_string, if_exists = if_ex_val, index=False) 
        except Exception as e:
            print(e)
            print("Data load failed: " + table_name)   
            break
        break    
    

In [14]:
# Initial load of the historical data

def main():
    
    # Investment instruments
    inv_instruments = ['URTH', 'EXW1.DE', 'CSSMI.SW', 'EXXT.DE', 'GC=F', 'SI=F']
    labels={'GC=F':'gold','SI=F':'silver','URTH':'msci_world', 'EXW1.DE':'euro_stoxx', 'CSSMI.SW':'smi', 'EXXT.DE':'nasdaq'}
    
    # set timeframe
    now = datetime(2022, 3, 31)
    then = datetime(2022, 2, 1)

    # set method of function df.to_sql()
    method = 'replace'

    # loop for all investment instruments
    for ins in inv_instruments:
        label = labels[ins]
        fetch_load_data(ins, label, then, now, method)   

In [15]:
# execute function
if __name__ == "__main__":
    main()

table_yf_testmsci_world
table_yf_testeuro_stoxx
table_yf_testsmi
table_yf_testnasdaq
table_yf_testgold
table_yf_testsilver
